In [1]:
# Environment detection and system information
import os
import sys
import platform
import subprocess
from datetime import datetime

# Define function to check if we're in a RunPod environment
def is_runpod():
    """Check if we're running in a RunPod environment"""
    return (
        os.environ.get("RUNPOD_POD_ID") is not None or 
        "A100" in os.environ.get("GPU_NAME", "") or
        "H100" in os.environ.get("GPU_NAME", "")
    )

# Function to detect CUDA version in the environment
def detect_cuda_version():
    """Detect the CUDA version available in the environment"""
    try:
        # Try to get CUDA version from nvcc
        cuda_version = subprocess.check_output("nvcc --version | grep release", shell=True).decode('utf-8').strip()
        if "release" in cuda_version:
            # Extract the version number (e.g., "11.8" from "release 11.8")
            version_parts = cuda_version.split("release ")[1].split(",")[0].split(".")
            if len(version_parts) >= 2:
                major = int(version_parts[0])
                minor = int(version_parts[1])
                return f"{major}.{minor}"
    except:
        pass
    
    # If nvcc detection fails, try nvidia-smi
    try:
        smi_output = subprocess.check_output("nvidia-smi", shell=True).decode('utf-8')
        for line in smi_output.split('\n'):
            if "CUDA Version:" in line:
                cuda_version = line.split("CUDA Version:")[1].strip()
                return cuda_version
    except:
        pass
    
    return None

# Display basic system information
print(f"📋 Installation Environment Check")
print(f"🕒 Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💻 System: {platform.system()} {platform.release()}")
print(f"🐍 Python version: {platform.python_version()}")
print(f"📂 Working directory: {os.getcwd()}")

# Check and display GPU information
try:
    if is_runpod():
        print(f"☁️ RunPod environment detected!")
        
        # Get GPU information if available
        try:
            gpu_info = subprocess.check_output("nvidia-smi", shell=True).decode('utf-8')
            gpu_lines = [line for line in gpu_info.split('\n') if "NVIDIA" in line and not "Driver" in line]
            if gpu_lines:
                print(f"🖥️ GPU detected: {gpu_lines[0].strip()}")
            
            # Get CUDA version
            cuda_version_str = detect_cuda_version()
            if cuda_version_str:
                print(f"⚡ CUDA Version: {cuda_version_str}")
                
                # Set an environment variable with the detected CUDA version
                # This will be used in subsequent cells to determine compatibility
                os.environ["DETECTED_CUDA_VERSION"] = cuda_version_str
                
                # Add new check for H100 GPU with CUDA compatibility hint
                if "H100" in gpu_info:
                    if cuda_version_str.startswith("11"):
                        print("ℹ️ H100 GPU detected with CUDA 11.x - CUDA 12.x would be more optimal but 11.x will work")
                    elif cuda_version_str.startswith("12"):
                        print("✅ H100 GPU detected with optimal CUDA 12.x")
        except Exception as e:
            print(f"⚠️ GPU information not available: {str(e)}")
    else:
        print("💻 Local environment detected")
except Exception as e:
    print(f"❌ Error checking environment: {e}")

📋 Installation Environment Check
🕒 Current time: 2025-03-23 15:43:59
💻 System: Linux 6.8.0-50-generic
🐍 Python version: 3.10.12
📂 Working directory: /workspace/notebooks
☁️ RunPod environment detected!
🖥️ GPU detected: |   0  NVIDIA H100 NVL                On  |   00000000:01:00.0 Off |                    0 |
⚡ CUDA Version: 11.8
ℹ️ H100 GPU detected with CUDA 11.x - CUDA 12.x would be more optimal but 11.x will work


Core PyTorch Installation

In [2]:
# Core PyTorch Installation
import os
import sys
import subprocess
import time

# Skip if not in RunPod environment
if not is_runpod():
    print("💻 Local environment - skipping PyTorch installation")
    
    # Try to import and display PyTorch version if available
    try:
        import torch
        print(f"✅ PyTorch {torch.__version__} already installed")
        print(f"🔍 CUDA available: {torch.cuda.is_available()}")
        if torch.cuda.is_available():
            print(f"🖥️ GPU: {torch.cuda.get_device_name(0)}")
    except ImportError:
        print("⚠️ PyTorch not found in local environment")
else:
    print("\n🚀 Installing PyTorch with CUDA support")
    print("========================================")
    
    # Use detected CUDA version from the previous cell if available
    cuda_version = os.environ.get("DETECTED_CUDA_VERSION", "11.8")  # Default to CUDA 11.8 if not detected
    
    # Override with explicitly supported versions to ensure compatibility
    if cuda_version.startswith("11"):
        cuda_version = "11.8"  # Use 11.8 for any 11.x version
    elif cuda_version.startswith("12"):
        cuda_version = "12.1"  # Use 12.1 for any 12.x version
    
    print(f"🔍 Installing PyTorch for CUDA {cuda_version}")
    
    # Set PyTorch version to 2.1.0 for compatibility with newer Transformers
    # and map CUDA version to correct URL
    if cuda_version.startswith("12"):
        torch_url = "https://download.pytorch.org/whl/cu121"
    else:
        torch_url = "https://download.pytorch.org/whl/cu118"
    
    # Use PyTorch 2.1.0 which is needed for newer Transformers
    torch_version = "2.1.0"
    torchvision_version = "0.16.0"
    torchaudio_version = "2.1.0"
    
    try:
        print(f"⏳ Installing PyTorch {torch_version} with CUDA {cuda_version} (this may take a few minutes)...")
        subprocess.check_call([
            sys.executable, 
            "-m", 
            "pip", 
            "install", 
            f"torch=={torch_version}", 
            f"torchvision=={torchvision_version}", 
            f"torchaudio=={torchaudio_version}", 
            "--index-url", 
            torch_url,
            "--no-cache-dir"  # Added to avoid caching issues
        ])
        
        print(f"✅ PyTorch installation successful!")
        
        # Verify installation
        import torch
        print(f"✅ PyTorch {torch.__version__} installed with CUDA {torch.version.cuda}")
        print(f"🔍 CUDA available: {torch.cuda.is_available()}")
        if torch.cuda.is_available():
            print(f"🖥️ GPU: {torch.cuda.get_device_name(0)}")
            print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    
    except Exception as e:
        print(f"❌ PyTorch installation failed: {e}")
        print("⚠️ Proceeding with existing or basic PyTorch installation")
        
        try:
            # Try to verify installation even if subprocess failed
            import torch
            print(f"✅ Using existing PyTorch {torch.__version__} with CUDA {torch.version.cuda}")
            print(f"🔍 CUDA available: {torch.cuda.is_available()}")
            if torch.cuda.is_available():
                print(f"🖥️ GPU: {torch.cuda.get_device_name(0)}")
                print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        except ImportError:
            print("❌ PyTorch not available")


🚀 Installing PyTorch with CUDA support
🔍 Installing PyTorch for CUDA 11.8
⏳ Installing PyTorch 2.1.0 with CUDA 11.8 (this may take a few minutes)...
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 255.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 345.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 394.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 271.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: triton
    Found existing installation: triton 2.0.0
    Uninstalling triton-2.0.0:
      Successfully uninstalled triton-2.0.0
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.2+cu118
    Uninstalling torchvision-0.15.2+cu1

✅ PyTorch installation successful!
✅ PyTorch 2.1.0+cu118 installed with CUDA 11.8
🔍 CUDA available: True
🖥️ GPU: NVIDIA H100 NVL
💾 GPU Memory: 99.98 GB


Development Tools

In [11]:
# Development Tools Installation
import os
import sys
import subprocess
import time

# Skip if not in RunPod environment
if not is_runpod():
    print("💻 Local environment - skipping development tools installation")
else:
    print("\n🛠️ Installing Development Tools")
    print("===============================")
    
    # List of development tools to install
    dev_tools = [
        {"name": "Ninja build system", "cmd": ["apt-get", "update", "-y"]},
        {"name": "Ninja build system", "cmd": ["apt-get", "install", "-y", "ninja-build"]},
        {"name": "Build essentials", "cmd": ["apt-get", "install", "-y", "build-essential"]},
        {"name": "CMake", "cmd": [sys.executable, "-m", "pip", "install", "cmake", "--upgrade"]},
        {"name": "Setuptools", "cmd": [sys.executable, "-m", "pip", "install", "setuptools", "wheel", "--upgrade"]}
    ]
    
    # Install each tool
    for tool in dev_tools:
        try:
            print(f"⏳ Installing {tool['name']}...")
            start_time = time.time()
            
            subprocess.check_call(tool["cmd"])
            
            print(f"✅ {tool['name']} installed successfully! (Took {(time.time() - start_time):.1f}s)")
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to install {tool['name']}: {e}")
            if "apt-get" in tool["cmd"]:
                print("⚠️ This may be due to missing sudo privileges in RunPod environment")
                print("⚠️ Continuing with installation - some packages may fail to build")
    
    # Verify Ninja installation
    try:
        ninja_version = subprocess.check_output(["ninja", "--version"], stderr=subprocess.STDOUT).decode('utf-8').strip()
        print(f"✅ Ninja build system verified: version {ninja_version}")
    except:
        print("⚠️ Ninja build system not available in PATH")
        print("⚠️ Some packages like flash-attention may fail to build")


🛠️ Installing Development Tools
⏳ Installing Ninja build system...
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Reading package lists...
✅ Ninja build system installed successfully! (Took 1.3s)
⏳ Installing Ninja build system...
Reading package lists...
Building dependency tree...
Reading state information...
ninja-build is already the newest version (1.10.1-1).
0 upgraded, 0 newly installed, 0 to remove and 129 not upgraded.
✅ Ninja build system installed successfully! (Took 0.8s)
⏳ Installing Build essentials...
Reading package lists...
Building dependency tree...
Reading state information...
build-essential is alread


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


✅ CMake installed successfully! (Took 1.2s)
⏳ Installing Setuptools...
✅ Setuptools installed successfully! (Took 1.2s)
✅ Ninja build system verified: version 1.10.1



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


Optional Accelerators 

In [4]:
# Optional Accelerators Installation (flash-attention)
import os
import sys
import subprocess
import time

# Configuration - set to False to skip flash-attention
INSTALL_FLASH_ATTENTION = True

# Skip if not in RunPod environment or if disabled
if not is_runpod():
    print("💻 Local environment - skipping accelerator installation")
elif not INSTALL_FLASH_ATTENTION:
    print("⏭️ Flash-attention installation skipped by configuration")
else:
    print("\n⚡ Installing Optional Accelerators")
    print("==================================")
    
    print("🔍 Installing flash-attn for optimized attention...")
    start_time = time.time()
    
    try:
        # Use Popen to see continuous output
        print("⏳ Building flash-attention (this may take 5-15 minutes):")
        
        # Setup process for verbose output
        process = subprocess.Popen(
            [sys.executable, "-m", "pip", "install", "flash-attn", "--no-build-isolation", "-v"],
            stdout=subprocess.PIPE, 
            stderr=subprocess.STDOUT,
            universal_newlines=True
        )
        
        # Print output in real-time with a timestamp
        last_update = time.time()
        for line in process.stdout:
            # Only print every few seconds to avoid flooding the output
            current_time = time.time()
            if current_time - last_update > 5:  # Print update every 5 seconds
                elapsed = current_time - start_time
                print(f"⏳ Still building flash-attention... ({elapsed:.1f}s elapsed)")
                last_update = current_time
        
        # Wait for process to finish and check return code
        returncode = process.wait()
        if returncode != 0:
            raise subprocess.CalledProcessError(returncode, "flash-attn installation")
        
        print(f"✅ Flash-attention installed successfully! (Took {(time.time() - start_time):.1f}s)")
        
        # Test if it can be imported
        try:
            subprocess.check_call([
                sys.executable,
                "-c",
                "import flash_attn; print(f'✅ Flash-attention version: {flash_attn.__version__}')"
            ])
        except:
            print("⚠️ Flash-attention installed but import failed")
            
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install flash-attention: {e}")
        print("⚠️ Will fall back to standard attention implementation (slightly slower)")


⚡ Installing Optional Accelerators
🔍 Installing flash-attn for optimized attention...
⏳ Building flash-attention (this may take 5-15 minutes):
⏳ Still building flash-attention... (6.1s elapsed)


KeyboardInterrupt: 

In [ ]:
Core Dependencies

In [3]:
# Core Dependencies Installation (transformers, accelerate, numpy)
import os
import sys
import subprocess
import time

# Skip if not in RunPod environment
if not is_runpod():
    print("💻 Local environment - skipping core dependencies installation")
    
    # Try to import and display versions if available
    try:
        import transformers
        import accelerate
        import numpy as np
        print(f"✅ transformers {transformers.__version__}, accelerate {accelerate.__version__}, numpy {np.__version__} already installed")
    except ImportError as e:
        print(f"⚠️ Some core dependencies not found: {e}")
else:
    print("\n📦 Installing Core Dependencies")
    print("=============================")
    
    # First uninstall potentially conflicting packages to avoid version issues
    print("🧹 Removing potentially conflicting packages...")
    for package in ["transformers", "accelerate", "numpy"]:  # Don't remove PyTorch
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", package])
            print(f"  ✓ Removed {package}")
        except:
            print(f"  ✓ No existing {package} to remove")
    
    # Now install core dependencies with versions compatible with PyTorch 2.1.0
    core_packages = [
        {"name": "NumPy", "package": "numpy==1.24.3"},
        {"name": "Transformers", "package": "transformers>=4.45.0"},  # Latest version
        {"name": "Accelerate", "package": "accelerate>=0.26.0"},
        {"name": "SentencePiece", "package": "sentencepiece==0.1.99"}
    ]
    
    # Install each core package
    for package in core_packages:
        try:
            print(f"⏳ Installing {package['name']}...")
            start_time = time.time()
            
            subprocess.check_call([
                sys.executable, "-m", "pip", "install", package["package"], "--no-cache-dir"
            ])
            
            print(f"✅ {package['name']} installed successfully! (Took {(time.time() - start_time):.1f}s)")
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to install {package['name']}: {e}")
    
    # Verify core installations
    print("\n🔍 Verifying core dependencies:")
    try:
        import numpy as np
        print(f"✅ NumPy {np.__version__}")
    except ImportError:
        print("❌ NumPy import failed")
    
    try:
        import torch
        print(f"✅ PyTorch {torch.__version__}")
    except ImportError:
        print("❌ PyTorch import failed")
    
    try:
        import transformers
        print(f"✅ Transformers {transformers.__version__}")
        
        # Test specific model classes needed for Pixtral
        try:
            from transformers import LlavaForConditionalGeneration, AutoProcessor
            print(f"✅ Successfully imported LlavaForConditionalGeneration")
        except ImportError as e:
            print(f"❌ Failed to import LlavaForConditionalGeneration: {e}")
            print("⚠️ This may affect Pixtral-12B model loading")
    except ImportError as e:
        print(f"❌ Transformers import failed: {e}")
    
    try:
        import accelerate
        print(f"✅ Accelerate {accelerate.__version__}")
    except ImportError:
        print("❌ Accelerate import failed")


📦 Installing Core Dependencies
🧹 Removing potentially conflicting packages...
Found existing installation: transformers 4.50.0
Uninstalling transformers-4.50.0:
  Successfully uninstalled transformers-4.50.0


  ✓ Removed transformers
Found existing installation: accelerate 1.5.2
Uninstalling accelerate-1.5.2:
  Successfully uninstalled accelerate-1.5.2
  ✓ Removed accelerate


Found existing installation: numpy 1.24.3
Uninstalling numpy-1.24.3:
  Successfully uninstalled numpy-1.24.3
  ✓ Removed numpy
⏳ Installing NumPy...


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 325.4 MB/s eta 0:00:00 0:00:01
✅ NumPy installed successfully! (Took 2.4s)
⏳ Installing Transformers...



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 18.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


✅ Accelerate installed successfully! (Took 1.4s)
⏳ Installing SentencePiece...



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


✅ SentencePiece installed successfully! (Took 1.1s)

🔍 Verifying core dependencies:
✅ NumPy 1.24.3
✅ PyTorch 2.1.0+cu118
✅ Transformers 4.50.0
✅ Successfully imported LlavaForConditionalGeneration
✅ Accelerate 1.5.2


Quantization Dependencies

In [4]:
# Quantization Dependencies Installation (bitsandbytes)
import os
import sys
import subprocess
import time

# Skip if not in RunPod environment
if not is_runpod():
    print("💻 Local environment - skipping quantization dependencies installation")
    
    # Try to import and display version if available
    try:
        import bitsandbytes as bnb
        print(f"✅ bitsandbytes {bnb.__version__} already installed")
    except ImportError:
        print("⚠️ bitsandbytes not found in local environment")
else:
    print("\n🔢 Installing Quantization Dependencies")
    print("=====================================")
    
    # Install bitsandbytes dependencies first
    bitsandbytes_dependencies = [
        {"name": "SciPy", "package": "scipy>=1.9.0"},
        {"name": "Typing Extensions", "package": "typing_extensions==4.9.0"}
    ]
    
    # Install each dependency
    for dependency in bitsandbytes_dependencies:
        try:
            print(f"⏳ Installing {dependency['name']}...")
            start_time = time.time()
            
            subprocess.check_call([
                sys.executable, "-m", "pip", "install", dependency["package"]
            ])
            
            print(f"✅ {dependency['name']} installed successfully! (Took {(time.time() - start_time):.1f}s)")
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to install {dependency['name']}: {e}")
    
    # Install bitsandbytes for quantization
    print("\n⏳ Installing bitsandbytes for model quantization...")
    start_time = time.time()
    
    try:
        # First ensure we don't have an existing installation
        subprocess.check_call([
            sys.executable, "-m", "pip", "uninstall", "-y", "bitsandbytes"
        ])
        print("  ✓ Removed existing bitsandbytes installation")
    except:
        print("  ✓ No existing bitsandbytes to remove")
    
    # Now install bitsandbytes
    try:
        subprocess.check_call([
            sys.executable, "-m", "pip", "install", "bitsandbytes==0.41.1", "--no-cache-dir"
        ])
        print(f"✅ bitsandbytes installed successfully! (Took {(time.time() - start_time):.1f}s)")
        
        # Test bitsandbytes functionality
        print("\n🔍 Testing bitsandbytes CUDA functionality:")
        try:
            test_code = """
import bitsandbytes as bnb
import torch
print("✅ bitsandbytes loaded successfully")
linear8bit = bnb.nn.Linear8bitLt(10, 10)
print("✅ 8-bit Linear layer created successfully")
"""
            subprocess.check_call([sys.executable, "-c", test_code])
            print("✅ bitsandbytes CUDA functionality verified!")
        except subprocess.CalledProcessError as e:
            print(f"❌ CUDA functionality test failed: {e}")
            print("⚠️ 8-bit quantization may not be available")
            
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install bitsandbytes: {e}")
        print("⚠️ Model quantization won't be available")


🔢 Installing Quantization Dependencies
⏳ Installing SciPy...



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


✅ SciPy installed successfully! (Took 1.0s)
⏳ Installing Typing Extensions...



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


✅ Typing Extensions installed successfully! (Took 1.1s)

⏳ Installing bitsandbytes for model quantization...
Found existing installation: bitsandbytes 0.41.1
Uninstalling bitsandbytes-0.41.1:
  Successfully uninstalled bitsandbytes-0.41.1
  ✓ Removed existing bitsandbytes installation


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 222.9 MB/s eta 0:00:0000:0100:01
✅ bitsandbytes installed successfully! (Took 3.1s)

🔍 Testing bitsandbytes CUDA functionality:



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Traceback (most recent call last):
  File "<string>", line 4, in <module>
AttributeError: module 'bitsandbytes' has no attribute '__version__'


❌ CUDA functionality test failed: Command '['/usr/bin/python', '-c', '\nimport bitsandbytes as bnb\nimport torch\nprint(f"bitsandbytes version: {bnb.__version__}")\nlinear8bit = bnb.nn.Linear8bitLt(10, 10)\nprint("✅ 8-bit Linear layer created successfully")\n']' returned non-zero exit status 1.
⚠️ 8-bit quantization may not be available


Additional Packages

In [7]:
# Additional Packages Installation
import os
import sys
import subprocess
import time

# Skip if not in RunPod environment
if not is_runpod():
    print("💻 Local environment - skipping additional packages installation")
else:
    print("\n📚 Installing Additional Utility Packages")
    print("=======================================")
    
    # List of additional packages to install
    additional_packages = [
        {"name": "tqdm", "package": "tqdm==4.66.1"},
        {"name": "Pillow", "package": "pillow==10.0.0"},
        {"name": "Matplotlib", "package": "matplotlib==3.7.2"},
        {"name": "Pandas", "package": "pandas==2.0.3"},
        {"name": "Levenshtein", "package": "python-Levenshtein==0.21.1"},
        {"name": "PyYAML", "package": "pyyaml"},
        {"name": "Packaging", "package": "packaging"}
    ]
    
    # Install each package
    for package in additional_packages:
        try:
            print(f"⏳ Installing {package['name']}...")
            start_time = time.time()
            
            subprocess.check_call([
                sys.executable, "-m", "pip", "install", package["package"]
            ])
            
            print(f"✅ {package['name']} installed successfully! (Took {(time.time() - start_time):.1f}s)")
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to install {package['name']}: {e}")
            print(f"⚠️ Continuing with other packages...")


📚 Installing Additional Utility Packages
⏳ Installing tqdm...



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


✅ tqdm installed successfully! (Took 1.0s)
⏳ Installing Pillow...



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


✅ Pillow installed successfully! (Took 1.1s)
⏳ Installing Matplotlib...


...


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


✅ Levenshtein installed successfully! (Took 1.0s)
⏳ Installing PyYAML...



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


✅ PyYAML installed successfully! (Took 1.0s)
⏳ Installing Packaging...
✅ Packaging installed successfully! (Took 1.0s)



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


Verification

In [5]:
# Verification of Critical Components
import os
import sys
import subprocess
import time

print("\n🔍 Verifying Critical Components")
print("==============================")

# Check PyTorch and CUDA
print("\n📊 PyTorch and CUDA Status:")
try:
    import torch
    print(f"✅ PyTorch version: {torch.__version__}")
    
    if torch.cuda.is_available():
        print(f"✅ CUDA available: version {torch.version.cuda}")
        print(f"✅ GPU device: {torch.cuda.get_device_name(0)}")
        print(f"✅ GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        
        # Test basic CUDA operations
        print("🧪 Testing CUDA tensor operations...")
        try:
            start_time = time.time()
            # Create a CUDA tensor and perform operations
            x = torch.randn(1000, 1000, device="cuda")
            y = torch.randn(1000, 1000, device="cuda")
            z = torch.matmul(x, y)
            torch.cuda.synchronize()  # Wait for operation to complete
            print(f"✅ CUDA tensor operations successful (Took {(time.time() - start_time):.3f}s)")
        except Exception as e:
            print(f"❌ CUDA tensor operations failed: {e}")
    else:
        print("⚠️ CUDA not available - GPU acceleration disabled")
except ImportError:
    print("❌ PyTorch not installed")

# Check Transformers and model loading capabilities
print("\n🤖 Transformers Status:")
try:
    import transformers
    print(f"✅ Transformers version: {transformers.__version__}")
    
    # Verify model loading capabilities
    print("🧪 Testing model class imports...")
    try:
        subprocess.check_call([
            sys.executable,
            "-c",
            "from transformers import LlavaForConditionalGeneration, AutoProcessor; print('✅ Successfully imported vision-language model classes')"
        ])
    except subprocess.CalledProcessError:
        print("❌ Failed to import vision-language model classes")
        print("⚠️ Attempting alternative model loading approach...")
        try:
            subprocess.check_call([
                sys.executable,
                "-c",
                "from transformers import AutoModelForVision2Seq, AutoProcessor; print('✅ Successfully imported AutoModelForVision2Seq as fallback')"
            ])
        except subprocess.CalledProcessError:
            print("❌ Failed to import alternative model classes")
except ImportError:
    print("❌ Transformers not installed")

# Check quantization support
print("\n🔢 Quantization Support:")
try:
    import bitsandbytes as bnb
    print("✅ bitsandbytes loaded successfully")
    
    # Test if 8-bit quantization works
    try:
        subprocess.check_call([
            sys.executable,
            "-c",
            "import bitsandbytes as bnb; import torch; linear8bit = bnb.nn.Linear8bitLt(10, 10); print('✅ 8-bit quantization supported')"
        ])
    except subprocess.CalledProcessError:
        print("❌ 8-bit quantization not working")
except ImportError:
    print("⚠️ bitsandbytes not installed - quantization will be unavailable")

# Check flash-attention
print("\n⚡ Accelerator Status:")
try:
    subprocess.check_call([
        sys.executable,
        "-c",
        "import flash_attn; print(f'✅ flash-attention version: {flash_attn.__version__}')"
    ], stderr=subprocess.DEVNULL)
except:
    print("⚠️ flash-attention not installed - using standard attention")

# Check additional packages
print("\n📚 Additional Packages:")
packages_to_check = [
    "numpy", "pandas", "matplotlib", "PIL", "tqdm", "yaml"
]

for package in packages_to_check:
    try:
        if package == "PIL":
            subprocess.check_call([
                sys.executable,
                "-c",
                f"from {package} import Image; print('✅ {package} available')"
            ], stderr=subprocess.DEVNULL)
        else:
            subprocess.check_call([
                sys.executable,
                "-c",
                f"import {package}; print('✅ {package} available')"
            ], stderr=subprocess.DEVNULL)
    except:
        print(f"⚠️ {package} not installed or not working")


🔍 Verifying Critical Components

📊 PyTorch and CUDA Status:
✅ PyTorch version: 2.1.0+cu118
✅ CUDA available: version 11.8
✅ GPU device: NVIDIA H100 NVL
✅ GPU memory: 99.98 GB
🧪 Testing CUDA tensor operations...
✅ CUDA tensor operations successful (Took 0.686s)

🤖 Transformers Status:
✅ Transformers version: 4.50.0
🧪 Testing model class imports...
✅ Successfully imported vision-language model classes

🔢 Quantization Support:
✅ bitsandbytes loaded successfully
✅ 8-bit quantization supported

⚡ Accelerator Status:
⚠️ flash-attention not installed - using standard attention

📚 Additional Packages:
✅ numpy available
✅ pandas available
✅ matplotlib available
✅ PIL available
✅ tqdm available
✅ yaml available


Environment Summary

In [ ]:
# Environment Summary
import os
import sys
import platform
from datetime import datetime
import subprocess

print("\n📋 Environment Summary")
print("====================")
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# System information
print("\n💻 System Information:")
print(f"  OS: {platform.system()} {platform.release()}")
print(f"  Python: {platform.python_version()}")
print(f"  Directory: {os.getcwd()}")

# Environment type
env_type = "RunPod" if is_runpod() else "Local"
print(f"  Environment: {env_type}")

# GPU information
try:
    import torch
    if torch.cuda.is_available():
        print(f"\n🖥️ GPU Information:")
        print(f"  Device: {torch.cuda.get_device_name(0)}")
        print(f"  CUDA Version: {torch.version.cuda}")
        print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        
        # Additional GPU stats if nvidia-smi is available
        try:
            gpu_stats = subprocess.check_output("nvidia-smi --query-gpu=utilization.gpu,utilization.memory,temperature.gpu --format=csv,noheader", shell=True).decode('utf-8').strip()
            if gpu_stats:
                stats = gpu_stats.split(',')
                if len(stats) >= 3:
                    print(f"  GPU Utilization: {stats[0].strip()}")
                    print(f"  Memory Utilization: {stats[1].strip()}")
                    print(f"  Temperature: {stats[2].strip()}")
        except:
            pass  # Skip if nvidia-smi isn't available or fails
    else:
        print("\n⚠️ No GPU detected")
except ImportError:
    print("\n❌ PyTorch not installed - GPU information unavailable")

# Installed package versions
print("\n📦 Installed Packages:")
key_packages = [
    "torch", "transformers", "accelerate", "bitsandbytes", "numpy", 
    "pandas", "matplotlib", "sentencepiece", "tqdm", "pillow"
]

for package in key_packages:
    try:
        # Get package version
        version = subprocess.check_output(
            [sys.executable, "-c", f"import {package}; print({package}.__version__)"],
            stderr=subprocess.DEVNULL
        ).decode('utf-8').strip()
        print(f"  {package}: {version}")
    except:
        print(f"  {package}: Not installed")

# Flash attention status
try:
    fa_version = subprocess.check_output(
        [sys.executable, "-c", "import flash_attn; print(flash_attn.__version__)"],
        stderr=subprocess.DEVNULL
    ).decode('utf-8').strip()
    print(f"  flash-attn: {fa_version}")
except:
    print("  flash-attn: Not installed")

# Check model support
print("\n🤖 Model Support:")
model_classes = [
    "LlavaForConditionalGeneration", 
    "AutoModelForVision2Seq",
    "AutoProcessor"
]

for model_class in model_classes:
    try:
        subprocess.check_call(
            [sys.executable, "-c", f"from transformers import {model_class}"],
            stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL
        )
        print(f"  ✅ {model_class}")
    except:
        print(f"  ❌ {model_class}")

# Overall status
print("\n🚀 Environment Status:")
try:
    import torch
    import transformers
    
    if torch.cuda.is_available() and hasattr(transformers, "LlavaForConditionalGeneration"):
        print("✅ Environment is READY for vision-language model inference!")
    elif torch.cuda.is_available():
        print("⚠️ Environment has GPU support but may have issues with some model classes")
    else:
        print("⚠️ Environment lacks GPU acceleration - inference will be slow")
except:
    print("❌ Critical components missing - please review errors above")

print("\n✨ Environment setup report complete!")


📋 Environment Summary
📅 Date: 2025-03-23 15:47:04

💻 System Information:
  OS: Linux 6.8.0-50-generic
  Python: 3.10.12
  Directory: /workspace/notebooks
  Environment: RunPod

🖥️ GPU Information:
  Device: NVIDIA H100 NVL
  CUDA Version: 11.8
  Memory: 99.98 GB
  GPU Utilization: 0 %
  Memory Utilization: 0 %
  Temperature: 40

📦 Installed Packages:
  torch: 2.1.0+cu118
  transformers: 4.50.0
  accelerate: 1.5.2
